# 🧼 Data Cleaning Plan + Implementación 
### DogDayCare — Transformación de datos crudos a datos confiables
Este notebook combina: 
- **Plan estratégico de limpieza**
- **Implementación real del proceso de limpieza** 

El objetivo es transformar los datasets crudos en datos **consistentes, completos y listos para análisis**.

In [1]:
import pandas as pd
import numpy as np

In [2]:
bookings_raw = pd.read_csv("../data/raw/bookings_raw.csv")
customers_raw = pd.read_csv("../data/raw/customers_raw.csv")
payments_raw = pd.read_csv("../data/raw/payments_raw.csv")

# 1. Plan Estratégico de Limpieza 

Antes de limpiar, definimos **qué** se va a limpiar, **cómo**, y **por qué**.

## Objetivo general
Transformar los datasets crudos en datos:

- consistentes  
- normalizados  
- deduplicados  
- conectados  
- confiables  

## Problemas detectados (del Data Audit)
### Bookings
- Fechas en múltiples formatos  
- Horas inválidas  
- Campos nulos críticos  
- Duplicados  
- Valores categóricos inconsistentes  

### Customers
- Emails faltantes  
- Duplicados por email  
- Fechas inconsistentes  
- Booleanos mezclados  

### Payments
- Estados de pago no estandarizados  
- Montos como texto  
- Monedas mezcladas  
- Pagos sin reserva asociada  

## Fases del proceso de limpieza
1. **Normalización de formatos**  
2. **Corrección de valores inválidos**  
3. **Deduplicación**  
4. **Reconstrucción de relaciones**  
5. **Validación final**

A continuación implementamos cada fase paso a paso.


# 2. Fase 1 — Normalización de formatos

En esta fase convertimos:

- fechas → formato ISO  
- horas → formato 24h  
- montos → numéricos  
- booleanos → True/False  
- categorías → minúsculas y sin espacios  

Esto permite que los datos sean procesables.


In [7]:
from dateutil import parser

def parse_date(x):
    if pd.isna(x):
        return pd.NaT
    
    x = str(x).strip()
    
    try:
        # dateutil parser detecta automáticamente el formato correcto
        return parser.parse(x, dayfirst=True, fuzzy=True)
    except:
        return pd.NaT


In [8]:
bookings_raw["booking_date_clean"] = bookings_raw["booking_date"].apply(parse_date)
customers_raw["signup_date_clean"] = customers_raw["signup_date"].apply(parse_date)
payments_raw["paid_at_clean"] = payments_raw["paid_at"].apply(parse_date)

In [9]:
from dateutil import parser

def parse_time(x):
    if pd.isna(x):
        return pd.NaT
    
    x = str(x).strip().lower()
    
    # Reemplazar formatos comunes tipo "17.10" → "17:10"
    x = x.replace(".", ":")
    
    try:
        dt = parser.parse(x, fuzzy=True)
        return dt.time()
    except:
        return pd.NaT


In [10]:
bookings_raw["start_time_clean"] = bookings_raw["start_time"].apply(parse_time)
bookings_raw["end_time_clean"] = bookings_raw["end_time"].apply(parse_time)

In [11]:
import re

def extract_currency(x):
    if pd.isna(x):
        return np.nan
    
    x = str(x).upper().strip()
    
    # Buscar códigos comunes
    if "SEK" in x:
        return "SEK"
    if "EUR" in x or "€" in x:
        return "EUR"
    
    # Si la columna currency ya viene limpia
    if x in ["SEK", "EUR"]:
        return x
    
    return np.nan


In [12]:
def clean_amount(x):
    if pd.isna(x):
        return np.nan
    
    x = str(x).upper().strip()
    
    # Quitar moneda si está pegada al número
    x = x.replace("SEK", "").replace("EUR", "").replace("€", "")
    
    # Reemplazar coma por punto
    x = x.replace(",", ".")
    
    # Quitar espacios
    x = x.replace(" ", "")
    
    # Si queda vacío → NaN
    if x == "":
        return np.nan
    
    try:
        return float(x)
    except:
        return np.nan


In [13]:
payments_raw["currency_clean"] = payments_raw["currency"].apply(extract_currency)

# Si la moneda viene dentro del monto
payments_raw.loc[payments_raw["currency_clean"].isna(), "currency_clean"] = (
    payments_raw["amount_gross"].apply(extract_currency)
)

payments_raw["amount_gross_clean"] = payments_raw["amount_gross"].apply(clean_amount)
payments_raw["tax_amount_clean"] = payments_raw["tax_amount"].apply(clean_amount)
payments_raw["fee_amount_clean"] = payments_raw["fee_amount"].apply(clean_amount)
payments_raw["amount_net_clean"] = payments_raw["amount_net"].apply(clean_amount)


In [14]:
payments_raw[["amount_gross", "amount_gross_clean", "currency", "currency_clean"]].head(20)

,amount_gross,amount_gross_clean,currency,currency_clean
0,35,35.0,sek,SEK
1,20,20.0,€,EUR
2,60,60.0,sek,SEK
3,40,40.0,SEK,SEK
4,60,60.0,€,EUR
5,20,20.0,€,EUR
6,NaN,NaN,Sek,SEK
7,35,35.0,€,EUR
8,15,15.0,sek,SEK
9,15,15.0,Sek,SEK


In [15]:
def clean_bool(x):
    if pd.isna(x):
        return np.nan
    
    x = str(x).strip().lower()
    
    true_values = ["yes", "y", "true", "1", "si", "sí"]
    false_values = ["no", "n", "false", "0"]
    
    if x in true_values:
        return True
    if x in false_values:
        return False
    
    # Todo lo demás → NaN (no es booleano real)
    return np.nan


In [16]:
# Bookings
bookings_raw["is_cancelled_clean"] = bookings_raw["is_cancelled"].apply(clean_bool)
bookings_raw["is_repeat_customer_clean"] = bookings_raw["is_repeat_customer"].apply(clean_bool)

# Customers
customers_raw["marketing_opt_in_clean"] = customers_raw["marketing_opt_in"].apply(clean_bool)
customers_raw["waiver_signed_clean"] = customers_raw["waiver_signed"].apply(clean_bool)

# Payments
payments_raw["chargeback_flag_clean"] = payments_raw["chargeback_flag"].apply(clean_bool)

In [18]:
bookings_raw["booking_date_clean"] = bookings_raw["booking_date"].apply(parse_date)
customers_raw["signup_date_clean"] = customers_raw["signup_date"].apply(parse_date)
payments_raw["paid_at_clean"] = payments_raw["paid_at"].apply(parse_date)
payments_raw["refunded_at_clean"] = payments_raw["refunded_at"].apply(parse_date)

In [19]:
payments_raw["is_refunded"] = payments_raw["refunded_at_clean"].notna()

In [20]:
bookings_raw["has_cancel_reason"] = bookings_raw["cancel_reason"].notna()

In [22]:
bookings_raw[["is_cancelled", "is_cancelled_clean"]].head(20)

,is_cancelled,is_cancelled_clean
0,0,False
1,yes,True
2,NaN,NaN
3,1,True
4,NaN,NaN
5,No,False
6,NaN,NaN
7,1,True
8,NaN,NaN
9,1,True


In [23]:
customers_raw[["marketing_opt_in", "marketing_opt_in_clean"]].head(20)

,marketing_opt_in,marketing_opt_in_clean
0,1,True
1,no,False
2,no,False
3,no,False
4,FALSE,False
5,Yes,True
6,no,False
7,FALSE,False
8,N,False
9,Y,True


In [24]:
def clean_category(x):
    if pd.isna(x):
        return np.nan
    
    x = str(x).strip().lower()
    
    # Normalizar espacios
    x = " ".join(x.split())
    
    # Convertir valores vacíos a NaN
    if x in ["", "nan", "none", "null", "n/a"]:
        return np.nan
    
    return x


In [25]:
bookings_raw["service_clean"] = bookings_raw["service"].apply(clean_category)
bookings_raw["service_code_clean"] = bookings_raw["service_code"].apply(clean_category)
bookings_raw["location_clean"] = bookings_raw["location"].apply(clean_category)
bookings_raw["channel_clean"] = bookings_raw["channel"].apply(clean_category)
bookings_raw["source_system_clean"] = bookings_raw["source_system"].apply(clean_category)
bookings_raw["dog_gender_clean"] = bookings_raw["dog_gender"].apply(clean_category)
bookings_raw["temperament_clean"] = bookings_raw["temperament"].apply(clean_category)

In [26]:
customers_raw["country_clean"] = customers_raw["country"].apply(clean_category)
customers_raw["city_clean"] = customers_raw["city"].apply(clean_category)
customers_raw["preferred_channel_clean"] = customers_raw["preferred_channel"].apply(clean_category)
customers_raw["customer_type_clean"] = customers_raw["customer_type"].apply(clean_category)
customers_raw["risk_flag_clean"] = customers_raw["risk_flag"].apply(clean_category)
customers_raw["language_clean"] = customers_raw["language"].apply(clean_category)


In [27]:
payments_raw["status_clean"] = payments_raw["status"].apply(clean_category)
payments_raw["payment_method_clean"] = payments_raw["payment_method"].apply(clean_category)
payments_raw["gateway_clean"] = payments_raw["gateway"].apply(clean_category)
payments_raw["country_of_card_clean"] = payments_raw["country_of_card"].apply(clean_category)
payments_raw["card_brand_clean"] = payments_raw["card_brand"].apply(clean_category)

In [28]:
service_map = {
    "daycare": "daycare",
    "day care": "daycare",
    "dc": "daycare",
    
    "nail trim": "nail_trim",
    "nailtrim": "nail_trim",
    "nt": "nail_trim",
    
    "vet shuttle": "vet_shuttle",
    "vetshuttle": "vet_shuttle",
    "vs": "vet_shuttle",
    
    "boarding": "boarding",
    "bd": "boarding",
    
    "training": "training",
    "tr": "training",
    
    "grooming": "grooming",
    "gr": "grooming"
}

bookings_raw["service_std"] = bookings_raw["service_clean"].map(service_map)

In [29]:
channel_map = {
    "ig": "instagram",
    "insta": "instagram",
    "instagram": "instagram",
    
    "phone": "phone",
    "walk-in": "walk_in",
    "walk in": "walk_in",
    
    "website": "website",
    "partner": "partner",
    "email": "email"
}

bookings_raw["channel_std"] = bookings_raw["channel_clean"].map(channel_map)


In [30]:
gender_map = {
    "male": "male",
    "m": "male",
    
    "female": "female",
    "f": "female",
    
    "unknown": "unknown",
    "u": "unknown",
    "n/a": "unknown"
}

bookings_raw["dog_gender_std"] = bookings_raw["dog_gender_clean"].map(gender_map)


In [31]:
bookings_raw[["service", "service_clean", "service_std"]].head(20)


,service,service_clean,service_std
0,nail trim,nail trim,nail_trim
1,Daycare,daycare,daycare
2,vet shuttle,vet shuttle,vet_shuttle
3,NaN,NaN,NaN
4,Nail Trim,nail trim,nail_trim
5,nail trim,nail trim,nail_trim
6,NAIL TRIM,nail trim,nail_trim
7,Vet Shuttle,vet shuttle,vet_shuttle
8,TRAINING,training,training
9,DAYCARE,daycare,daycare


In [32]:
bookings_raw[["channel", "channel_clean", "channel_std"]].head(20)


,channel,channel_clean,channel_std
0,IG,ig,instagram
1,IG,ig,instagram
2,website,website,website
3,IG,ig,instagram
4,NaN,NaN,NaN
5,website,website,website
6,IG,ig,instagram
7,partner,partner,partner
8,instagram,instagram,instagram
9,phone,phone,phone


In [33]:
bookings_raw[["dog_gender", "dog_gender_clean", "dog_gender_std"]].head(20)

,dog_gender,dog_gender_clean,dog_gender_std
0,Unknown,unknown,unknown
1,NaN,NaN,NaN
2,M,m,male
3,male,male,male
4,U,u,unknown
5,M,m,male
6,male,male,male
7,NaN,NaN,NaN
8,Unknown,unknown,unknown
9,F,f,female


# 3. Fase 2 — Corrección de valores inválidos

En esta fase corregimos:

- horas imposibles  
- fechas inválidas  
- valores como `NULL`, `n/a`, `unknown`  
- emails incorrectos  
- valores numéricos mal formateados  
- categorías inconsistentes  

El objetivo es garantizar que los datos sean coherentes y utilizables antes de deduplicar o reconstruir relaciones.


In [34]:
bookings_raw["start_time_clean"] = bookings_raw["start_time"].apply(parse_time)
bookings_raw["end_time_clean"] = bookings_raw["end_time"].apply(parse_time)

In [35]:
invalid_values = ["nan", "null", "n/a", "none", "", " "]

for df in [bookings_raw, customers_raw, payments_raw]:
    df.replace(invalid_values, np.nan, inplace=True)

# 4. Fase 3 — Deduplicación

Eliminamos duplicados sin perder información.

La deduplicación se realiza:

- por email en clientes  
- por booking_id en reservas  
- por payment_id en pagos  

Siempre conservamos el registro más reciente o más completo.


In [36]:
customers_clean = (
    customers_raw
    .sort_values("signup_date_clean")
    .drop_duplicates(subset=["email"], keep="last")
)

In [37]:
bookings_clean = bookings_raw.drop_duplicates(subset=["booking_id"], keep="last")
payments_clean = payments_raw.drop_duplicates(subset=["payment_id"], keep="last")

# 5. Fase 4 — Reconstrucción de relaciones

En esta fase reconstruimos las relaciones entre:

- **Pagos ↔ Reservas**  
- **Reservas ↔ Clientes**

Esto permite:

- detectar pagos huérfanos  
- detectar reservas sin cliente  
- preparar el modelo relacional final  


In [38]:
payments_clean = payments_clean.merge(
    bookings_clean[["booking_id", "customer_email"]],
    on="booking_id",
    how="left",
    suffixes=("", "_from_booking")
)

In [39]:
bookings_clean = bookings_clean.merge(
    customers_clean[["email", "customer_id"]],
    left_on="customer_email",
    right_on="email",
    how="left"
)

# 6. Fase 5 — Validación final

Verificamos:

- tipos correctos  
- duplicados eliminados  
- relaciones reconstruidas  
- valores válidos  
- columnas limpias y estandarizadas  

Este paso garantiza que los datos están listos para análisis, dashboards y modelos.


In [41]:
bookings_clean.info()

<class 'pandas.DataFrame'>
RangeIndex: 1197 entries, 0 to 1196
Data columns (total 53 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   booking_id                1197 non-null   int64         
 1   external_booking_ref      528 non-null    str           
 2   customer_name             1161 non-null   str           
 3   customer_email            1050 non-null   str           
 4   service                   1165 non-null   str           
 5   service_code              1077 non-null   str           
 6   booking_date              1171 non-null   str           
 7   start_time                1140 non-null   str           
 8   end_time                  1136 non-null   str           
 9   checkin_time              496 non-null    str           
 10  checkout_time             495 non-null    str           
 11  location                  1020 non-null   str           
 12  channel                   1009 

In [42]:
# columnas que queremos conservar
columns_to_keep = [
    "booking_id",
    "customer_email",
    "customer_id",
    "dog_name",
    "dog_breed",
    "dog_weight",
    "notes",
    
    # fechas y horas limpias
    "booking_date_clean",
    "start_time_clean",
    "end_time_clean",
    
    # booleanos limpios
    "is_cancelled_clean",
    "is_repeat_customer_clean",
    "has_cancel_reason",
    
    # categorías limpias
    "service_std",
    "service_code_clean",
    "location_clean",
    "channel_std",
    "source_system_clean",
    "dog_gender_std",
    "temperament_clean",
    
    # numéricos
    "price",
    "discount",
    "tax_rate"
]

# filtrar el dataframe
bookings_clean = bookings_clean[columns_to_keep]


In [43]:
customers_clean.info()

<class 'pandas.DataFrame'>
Index: 205 entries, 2 to 219
Data columns (total 37 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   customer_id              205 non-null    int64         
 1   full_name                199 non-null    str           
 2   email                    204 non-null    str           
 3   phone                    148 non-null    str           
 4   country                  195 non-null    str           
 5   city                     194 non-null    str           
 6   postal_code              129 non-null    str           
 7   address_line1            134 non-null    str           
 8   address_line2            103 non-null    str           
 9   signup_date              197 non-null    str           
 10  marketing_opt_in         192 non-null    str           
 11  preferred_channel        182 non-null    str           
 12  customer_type            177 non-null    str        

In [45]:
columns_to_keep_customers = [
    "customer_id",
    "full_name",
    "email",
    "phone",
    
    # ubicación
    "country_clean",
    "city_clean",
    "postal_code",
    "address_line1",
    "address_line2",
    
    # fechas y booleanos
    "signup_date_clean",
    "marketing_opt_in_clean",
    "waiver_signed_clean",
    
    # categorías limpias
    "preferred_channel_clean",
    "customer_type_clean",
    "risk_flag_clean",
    "language_clean",
    
    # negocio
    "lifetime_value",
    "pets_count",
    "primary_pet_name",
    "primary_pet_breed",
    "primary_pet_dob",
    "vet_provider",
    "vaccination_status",
    "notes"
]

customers_clean = customers_clean[columns_to_keep_customers]


In [44]:
payments_clean.info()

<class 'pandas.DataFrame'>
RangeIndex: 1344 entries, 0 to 1343
Data columns (total 40 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   payment_id                   1344 non-null   int64         
 1   booking_id                   1314 non-null   float64       
 2   customer_email               1147 non-null   str           
 3   amount_gross                 1214 non-null   str           
 4   currency                     1273 non-null   str           
 5   tax_amount                   514 non-null    float64       
 6   fee_amount                   547 non-null    str           
 7   amount_net                   341 non-null    float64       
 8   status                       1296 non-null   str           
 9   payment_method               1262 non-null   str           
 10  gateway                      1258 non-null   str           
 11  gateway_payment_ref          590 non-null    str      

In [46]:
columns_to_keep_payments = [
    "payment_id",
    "booking_id",
    "customer_email",
    
    # fechas
    "paid_at_clean",
    "refunded_at_clean",
    "is_refunded",
    
    # montos limpios
    "amount_gross_clean",
    "tax_amount_clean",
    "fee_amount_clean",
    "amount_net_clean",
    "currency_clean",
    
    # categorías limpias
    "status_clean",
    "payment_method_clean",
    "gateway_clean",
    "card_brand_clean",
    "country_of_card_clean"
]

payments_clean = payments_clean[columns_to_keep_payments]


In [47]:
import os

# Crear carpeta si no existe
clean_path = "../data/clean"
os.makedirs(clean_path, exist_ok=True)

# Guardar datasets limpios
bookings_clean.to_csv(f"{clean_path}/bookings_clean.csv", index=False)
customers_clean.to_csv(f"{clean_path}/customers_clean.csv", index=False)
payments_clean.to_csv(f"{clean_path}/payments_clean.csv", index=False)

print("Datasets limpios guardados correctamente.")


Datasets limpios guardados correctamente.
